<a href="https://colab.research.google.com/github/taichihaya/test/blob/master/2_BERT_livedoor_news_on_Google_Colaboratory_label_smoothing_r1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日本語BERTでlivedoorニュースを教師あり学習で分類

In [ ]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [ ]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：Livedoorニュースをダウンロードしてtsvファイル化

参考：https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb


In [ ]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2020-11-19 23:35:25--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  3.01MB/s    in 2.8s    

2020-11-19 23:35:29 (3.01 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['it-life-hack', 'dokujo-tsushin', 'smax', 'sports-watch', 'README.txt', 'topic-news', 'movie-enter', 'peachy', 'CHANGES.txt', 'kaden-channel', 'livedoor-homme']
カテゴリー数: 9
['it-life-hack', 'dokujo-tsushin', 'smax', 'sports-watch', 'topic-news', 'movie-enter', 'peachy', 'kaden-channel', 'livedoor-homme']


In [ ]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある


0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [ ]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [ ]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [ ]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])


従来、OSとOfficeソフトを中心に展開していたマイクロソフトが、方針転換を始めたようだ。Surfaceでハードウェアも提供するようになり、OSやUIも各デバイスで共通化するなど次世代に向けて動き始めた。■基本的にどのデバイスでも同じOS、同じユーザー体験へWindows Phone 8は従来のWindows Phoneとは変わり、OSの主要部分がPCのWindows 8と共通になるようだ。Surfaceのようなタブレット端末はCPU別に2つのOSが用意される。スマートフォンのようなARM向けのCPUを搭載したWindows RTと、従来のパソコンと同等のx86を搭載した通常のWindows 8だ。それぞれのOSは主要な部分が共通化し、タブレットのWindows RTとWindows Phone 8はCPUが同じなので、OS自体もかなり近い物になりそうだ。x86のCPUを採用するタブレットはキーボードがないだけで、OSはもちろんハードウェアも通常のパソコンと同じだ。それぞれのデバイスではMetro UIを使用し、ユーザー体験はそれぞれのデバイスで同じような物となる。これらはコンシューマー向けの製品だが、サーバー向けの次期OS Windows Server 2012でもMetroは使われるようで、マイクロソフト製品を使う限り、携帯電話からサーバーまで同等の使い勝手となる。■次世代に向けてパソコン以上に現在急成長しているのはスマートフォンと、タブレットだ。タブレット自体はマイクロソフトが2002年に提供したWindows XP Tablet Editionから本格的に対応している。当時はデバイス自体も未熟で、その後もいくつかの製品が登場したが、Windowsのタブレットはヒットしないままだった。そんな中で、アップルのiPadは2010年に登場するとたちまち大成功と言えるほど、世界中で人気商品となってしまった。iPadの成功はiPad専用アプリの存在など、様々な要因がありそうだが、利用しているユーザーの総合的な満足度が高い事が上げられる。ハードウェアやOS、アプリなどの総合的な満足度を上げるには、それらをすべて自社で提供できた方が有利だ。Androidタブレットもいくつかのハードウェアは提供されているものの、iPadほど売れているとはいえない。マイクロソフトはSurfa

In [ ]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()


(7376, 2)


,text,label
0,従来、OSとOfficeソフトを中心に展開していたマイクロソフトが、方針転換を始めたようだ。...,it-life-hack
1,2012年は多くに天体イベントがある天体イヤーだ。星や天体への関心高くなり、空を見上げる人も...,it-life-hack
2,最近では仕事での付き合いから、Facebookアカウントを交換することが増えてきた。周囲がや...,it-life-hack
3,英会話を学んでから何年も経ってしまうと覚えていたハズのフレーズすら出てこなくなりますよね。か...,it-life-hack
4,タイトルの写真、何かわかるだろうか？ルイ・ヴィトンでしょ？まあその通りなのだが、これケーキだ...,it-life-hack


In [ ]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()


{0: 'it-life-hack', 1: 'dokujo-tsushin', 2: 'smax', 3: 'sports-watch', 4: 'topic-news', 5: 'movie-enter', 6: 'peachy', 7: 'kaden-channel', 8: 'livedoor-homme'}
{'it-life-hack': 0, 'dokujo-tsushin': 1, 'smax': 2, 'sports-watch': 3, 'topic-news': 4, 'movie-enter': 5, 'peachy': 6, 'kaden-channel': 7, 'livedoor-homme': 8}


,text,label_index
0,従来、OSとOfficeソフトを中心に展開していたマイクロソフトが、方針転換を始めたようだ。...,0
1,2012年は多くに天体イベントがある天体イヤーだ。星や天体への関心高くなり、空を見上げる人も...,0
2,最近では仕事での付き合いから、Facebookアカウントを交換することが増えてきた。周囲がや...,0
3,英会話を学んでから何年も経ってしまうと覚えていたハズのフレーズすら出てこなくなりますよね。か...,0
4,タイトルの写真、何かわかるだろうか？ルイ・ヴィトンでしょ？まあその通りなのだが、これケーキだ...,0


In [ ]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,——この番組の見所は何だと思いますか？マット・ボマー：この番組の魅力は、究極的には、すごく型...,5
1,Android版ダウンロード：iPhone版「ガール」のためのニュースは「Peachy（才職...,6
2,21日発売の「週刊ポスト」は、今日にもメジャー移籍に向けたFA宣言を行うと目されていた東北楽...,3
3,マリリン・モンロー没後50年、初めて明かされる恋——。世界のセックス・シンボルと謳われたマリ...,5
4,7日、前大阪府知事・橋下徹氏のコメントが、ネット掲示板を中心に大きな話題になっている。共産党...,4


In [ ]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [ ]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


## 準備2：LivedoorニュースをBERT用のDataLoaderにする

Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [ ]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [ ]:
import torch
import torchtext  # torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')


In [ ]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [ ]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



In [ ]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


4426
1475
1475


In [ ]:
# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label)


tensor([    2,  6369,     9,     6,  2399,   530,     5,  7911, 28493, 28472,
         1358, 28523,  4172,    23, 20648,    24,    35,  6406,  3151,    11,
         1389,    15,    10,  5114,  9949,    11,   602,    15,    10,     8,
         7911, 28493, 28472,  1358,  1652,  4172, 22671, 20713,     9,     6,
          744,   126,     5,   120,    13,  2504,  3238,    35,  7223,     6,
          107, 18548,     6,   107, 17083,     6,   107,  7904, 28833,   245,
          140,  5842,    14,    31,    13,   625,     8,  7168,     6,  6369,
           14,   602,    15,    10,     5,     9,   213,  4859,     8,   107,
        21990,     7,   705,    14, 12932,    36,  5685,  1581,  2881,    38,
           57,  4859,     6,  6724,  5464,     7,  1277,    15,    10,    36,
         3593,    35, 23171,  2586,  2881,    38,    57,  4859,     6,   324,
          176,    13,   139,     6,  8938, 13856,   129,    64,    11, 11150,
        15988,   186,     7, 13258,    15,    10,    36, 16714, 

In [ ]:
# datasetの中身を文章に戻し、確認

print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))  # 文章
dic_id2cat[int(item.Label)]  # id


['[CLS]', 'ソニー', 'は', '、', '独自', '開発', 'の', 'バランス', '##ド', '##・', '##アー', '##マ', '##チュア', '(', 'BA', ')', '・', 'ドライバー', 'ユニット', 'を', '搭載', 'し', 'た', 'ヘッド', '##ホン', 'を', '発表', 'し', 'た', '。', 'バランス', '##ド', '##・', '##アー', '##マー', '##チュア', '##ドライバー', '##ユニット', 'は', '、', '今', 'まで', 'の', 'もの', 'と', '比べ', '小型', '・', '軽量', '、', '高', '感度', '、', '高', '解像度', '、', '高', '遮', '##音', '性', 'という', '特性', 'が', 'ある', 'と', 'いう', '。', '今回', '、', 'ソニー', 'が', '発表', 'し', 'た', 'の', 'は', '11', '機種', '。', '高', '音質', 'に', '音楽', 'が', '楽しめる', '「', 'リス', '##ニング', 'タイプ', '」', '4', '機種', '、', 'スマート', 'フォン', 'に', '対応', 'し', 'た', '「', 'マイク', '・', 'リモコン', '付き', 'タイプ', '」', '4', '機種', '、', '世界', '初', 'と', 'なる', '、', '電池', '収納', '部', 'など', 'を', 'ハウ', '##ジング', '内', 'に', '集約', 'し', 'た', '「', 'ノイズ', '##キャン', '##セ', '##リング', '##タイプ', '」', '1', '機種', '、', 'Blue', '##to', '##oth', '搭載', 'の', '「', 'ワイヤ', '##レス', 'タイプ', '」', '1', '機種', '、', '水', '##洗', '##い', '可能', 'な', '「', 'スポーツ', 'タイプ', '」', '1', '機種', 'と', '豊富', 'な', 'ラインナップ',

'kaden-channel'

In [ ]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


In [ ]:
# DataLoaderの動作確認 

batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)

## 準備3：BERTのクラス分類用のモデルを用意する

Huggingfaceさんのをそのまま使うのではなく、BERTのbaseだけ使い、残りは自分で実装する

In [ ]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
print(model)



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [ ]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # self.drop = nn.Dropout(p=0.3)

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換

        # vec_0 = self.drop(vec_0) # dropout

        output = self.cls(vec_0)  # 全結合層

        return output


In [ ]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


ネットワーク設定完了


## 準備4：BERTのファインチューニングの設定

In [ ]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [ ]:
import torch.nn.functional as F
def linear_combination(x, y, epsilon): 
    return epsilon*x + (1-epsilon)*y

def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon:float=0.2, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
    
    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return linear_combination(loss/n, nll, self.epsilon)

In [ ]:
class LabelSmoothingCrossEntropy2(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy2, self).__init__()
    def forward(self, x, target, smoothing=0.1):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + smoothing * smooth_loss
        return loss.mean()

In [ ]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])





# 損失関数の設定
# criterion = LabelSmoothingCrossEntropy() # 1回目 正解率：0.9261
# criterion = LabelSmoothingCrossEntropy() # 1回目 正解率：smothing0.2 0.9261
criterion = LabelSmoothingCrossEntropy2() # 2回目 正解率：
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

## 5. 訓練を実施

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [ ]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 4
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 0.6003 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 20 || Loss: 0.6290 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 30 || Loss: 0.5485 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 40 || Loss: 0.5535 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 50 || Loss: 0.5490 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 60 || Loss: 0.6041 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 70 || Loss: 0.6530 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 80 || Loss: 0.5406 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 90 || Loss: 0.5219 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 100 || Loss: 0.5476 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 110 || Loss: 0.5465 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 120 || Loss: 0.5420 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 130 || Loss: 0.5515 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 140 || Loss: 0.5762 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 150 || Loss: 0.5395 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 160 || Loss: 0.5334 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 170 || Lo

## テストデータでの性能を確認

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))


100%|██████████| 93/93 [00:55<00:00,  1.68it/s]

テストデータ1475個での正解率：0.9356


https://yoheikikuta.github.io/bert-japanese/

https://github.com/yoheikikuta/bert-japanese

の「BERT with SentencePiece for Japanese text.」

では、入力テキストにタイトルを含めていますが、今回はタイトルは除いています。

同様にタイトルを抜いている、[BERTを用いた日本語文書分類タスクの学習・ハイパーパラメータチューニングの実践例](https://medium.com/karakuri/bert%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%82%BF%E3%82%B9%E3%82%AF%E3%81%AE%E5%AD%A6%E7%BF%92-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E5%AE%9F%E8%B7%B5%E4%BE%8B-2fa5e4299b16)でも、正解率が92%ちょっととなっており、ほぼ同じ正解率が得られました。

以上。